# BÀI TẬP VỀ NHÀ.

## Bài tập số 1 (Chung)

In [1]:
from web3 import HTTPProvider
from web3 import Web3
from web3.middleware import geth_poa_middleware
import json
import operator

provider_url = "https://bsc-dataseed4.binance.org/"
web3 = Web3(HTTPProvider(provider_url))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

print(f"Success?: {web3.isConnected()}")

NUM_BLOCKS_1 = 100
NUM_BLOCKS_2 = 1000

Success?: True


### 1.Tính số transaction trung bình mỗi block của 100 blocks mới nhất

In [2]:
latest_block = web3.eth.block_number
num_txs = 0
for i in range(NUM_BLOCKS_1):
    block_txs = web3.eth.get_block_transaction_count(latest_block - i) 
    num_txs += block_txs
    # print each 10th block to track the progress
    if i % 10 == 0:
        print(f"Block {i} done")
avg_txs = num_txs / NUM_BLOCKS_1

result = {
    'fromBlock': latest_block - NUM_BLOCKS_1 + 1,
    'toBlock': latest_block,
    'averageTransactions': avg_txs
}
json_obj = json.dumps(result, indent=2)
with open('./Dap_an/genernal/genernal.json', 'w') as f:
    f.write(json_obj)

Block 0 done
Block 10 done
Block 20 done
Block 30 done
Block 40 done
Block 50 done
Block 60 done
Block 70 done
Block 80 done
Block 90 done


### 2. Cho địa chỉ token `0x7083609fCE4d1d8Dc0C979AAb8c869Ea2C873402`, dựa vào địa chỉ token và ERC_20 abi trong `abi/erc_20.json` 
Thực hiện các nhiệm vụ sau:<br>
    a. Thu thập Transfer event của token đó trong 1000 blocks gần nhất. <br>
    b. Tìm decimals của token.<br>
    c. Tính lượng cung (supply) của token đó. <br>
    d. Tính số dư (balance) token mới nhất của một địa chỉ ví thực hiện nhiều Transfer event nhất trong tập dữ liệu Transfer event đã crawl (gợi ý: địa chỉ này nằm trong trường `_from` của event). <br>
    e. Tính số dư token mới nhất trong địa chỉ ví là địa chỉ ví nhận trong nhiều Transfer event nhất trong tập dữ liệu Transfer event đã crawl (gợi ý: địa chỉ này nằm trong trường `_to` của event). <br>
    f. Tìm ký hiệu (symbol) và tên (name) của token.

Chú ý: Ngoài thông tin trả lời, cần ghi rõ lại start - end block mà mình thu thập dữ liệu, địa chỉ token và ví thu thập. Các giá trị lượng cung và số dư phải chia cho decimals của token.

In [3]:
#lấy cấu trúc dữ liệu của abi
with open("../abi/erc_20.json", "r") as f:
    abi = json.loads(f.read())
#Địa chỉ của valas  
token_address = "0x7083609fCE4d1d8Dc0C979AAb8c869Ea2C873402"
# Kiểm tra xem địa chỉ có đúng không
if not web3.isAddress(token_address):
    address = web3.toChecksumAddress(token_address)
#Sử dụng web3 thiết lập đối tượng contract để sử dụng các phương thức
contract = web3.eth.contract(abi=abi, address=token_address)
latest_block = web3.eth.blockNumber
start_block = latest_block - NUM_BLOCKS_2 + 1
end_block = latest_block
block_info = {
    'startBlock' : start_block,
    'endBlock' : end_block
}
print(f"Start block : {start_block}; End block : {end_block}")

Start block : 27847147; End block : 27848146


#### a.Thu thập Transfer event của token đó trong 1000 blocks gần nhất

In [4]:
transfer_events = contract.events.Transfer.createFilter(
    fromBlock = start_block , toBlock = end_block).get_all_entries()
transfers = list()
for event in transfer_events:
    transfers.append(json.loads(Web3.toJSON(event)))

result = dict()
result['blocksInfo'] = block_info
result['transferEvents'] = transfers
json_obj = json.dumps(result , indent = 2)
with open('./Dap_an/genernal/genernal2a.json', 'w') as f:
    f.write(json_obj)

#### b.Tìm decimals của token

In [5]:
decimals = contract.functions.decimals().call()
print(f"Token decimals : {decimals}")

result = {
    'decimals' : decimals,
}
json_obj = json.dumps(result,indent = 2)
with open('./Dap_an/genernal/genernal2b.json' , 'w') as f:
    f.write(json_obj)

Token decimals : 18


#### c.Tính lượng cung (supply) của token đó.

In [6]:
token_total_supply = contract.functions.totalSupply().call()
print(f"Token supply : {token_total_supply}")

result = {
    'totalSupply' : token_total_supply,
}

json_obj = json.dumps(result, indent=2)
with open('./Dap_an/genernal/genernal2c.json', 'w') as f:
    f.write(json_obj)

Token supply : 15999999991309562546802105


#### d.Tính số dư (balance) token mới nhất của một địa chỉ ví thực hiện nhiều Transfer event nhất trong tập dữ liệu Transfer event đã crawl (gợi ý: địa chỉ này nằm trong trường `_from` của event).

In [7]:
from_wallet_transfers = dict()
for transfer in transfers:
    from_wallet = transfer['args']['from']
    if from_wallet not in from_wallet_transfers:
        from_wallet_transfers[from_wallet] = 1
    else:
        from_wallet_transfers[from_wallet] += 1
# max from_wallet addr
max_from_wallet, max_frequency = max(from_wallet_transfers.items(), key=operator.itemgetter(1))
balance = contract.functions.balanceOf(max_from_wallet).call() / 10**decimals

print(f"Most transactions from {max_from_wallet} with balance: {balance}")

result = dict()
result['blocksInfo'] = block_info
result.update({
    'wallet': max_from_wallet,
    'maxFrequency': max_frequency,
    'balance': balance
})
json_obj = json.dumps(result, indent=2)
with open('./Dap_an/genernal/genernal2d.json', 'w') as f:
    f.write(json_obj)

Most transactions from 0xDd5bAd8f8b360d76d12FdA230F8BAF42fe0022CF with balance: 325223.6235284337


#### e.Tính số dư token mới nhất trong địa chỉ ví là địa chỉ ví nhận trong nhiều Transfer event nhất trong tập dữ liệu Transfer event đã crawl (gợi ý: địa chỉ này nằm trong trường `_to` của event).

In [8]:
to_wallet_transfers = dict()
for transfer in transfers:
    to_wallet = transfer['args']['to']
    if to_wallet not in to_wallet_transfers:
        to_wallet_transfers[to_wallet] = 1
    else:
        to_wallet_transfers[to_wallet] += 1

max_to_wallet, max_frequency = max(to_wallet_transfers.items(), key=operator.itemgetter(1))
balance = contract.functions.balanceOf(max_to_wallet).call() / 10**decimals

print(f"Most transactions to {max_to_wallet} with balance: {balance}")

result = dict()
result['blocksInfo'] = block_info
result.update({
    'wallet': max_to_wallet,
    'max_frequency': max_frequency,
    'balance': balance,
})
json_obj = json.dumps(result, indent=2)
with open('./Dap_an/genernal/genernal2e.json', 'w') as f:
    f.write(json_obj)

Most transactions to 0x39dD07ac211c33E796e6D89c4Df9481e0b60EcfE with balance: 4263.982548122644


#### f.Tìm ký hiệu (symbol) và tên (name) của token.

In [9]:
symbol =contract.functions.symbol().call()
name =contract.functions.name().call()
print(f"Symbol: {symbol}; Name: {name}")

result = {
    'symbol': symbol,
    'name': name
}

json_obj = json.dumps(result, indent=2)
with open('./Dap_an/genernal/genernal2f.json', 'w') as f:
    f.write(json_obj)

Symbol: DOT; Name: Polkadot Token
